In [1]:
import numpy as np
import numpy.linalg as npla

In [2]:
def nf2DualLP(filename):

    # assumes that first row is source and last is sink like in the question
    # edges will be numbered as if they are being read row-by-row left to right
    # vertices will be numbered by row

    nf = np.loadtxt(filename)
    for i in range(nf.shape[0]):
        nf[i, i] = 0
    numedges = np.count_nonzero(nf)
    numvertices = nf.shape[0] - 2     # non terminal vertices
    numslacks = numedges
    slack_counter = 0
    edge_counter = 0
    dual_constraints = np.zeros(
        (numedges, numedges + numvertices + numslacks + 1))
    obj = np.zeros(2 * numedges + numvertices)

    for i in range(numvertices + 2):
        for j in range(numvertices + 2):
            if nf[i, j] != 0:

                obj[edge_counter] = nf[i, j]

                if i == 0:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + j - 1] = 1
                    dual_constraints[edge_counter, numedges +
                                     numvertices + slack_counter] = -1
                    dual_constraints[edge_counter, -1] = 1
                    edge_counter += 1
                    slack_counter += 1

                elif j == numvertices + 1:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + i - 1] = -1
                    dual_constraints[edge_counter, numedges +
                                     numvertices + slack_counter] = -1
                    dual_constraints[edge_counter, -1] = 0
                    edge_counter += 1
                    slack_counter += 1

                else:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + i - 1] = -1
                    dual_constraints[edge_counter, numedges + j - 1] = 1
                    dual_constraints[edge_counter, numedges +
                                     numvertices + slack_counter] = -1
                    edge_counter += 1
                    slack_counter += 1

    sign_constraints = np.block([
        [np.eye(numedges), np.zeros((numedges, numvertices + numslacks + 1))],
        [np.zeros((numslacks, numedges + numvertices)),
         np.eye(numslacks), np.ones(numedges).reshape(1, numedges).T]
    ])

    LPMatrix = np.vstack((dual_constraints, sign_constraints))

    return LPMatrix, obj


def nf2PrimalLP(filename):
    nf = np.loadtxt(filename)
    for i in range(nf.shape[0]):
        nf[i, i] = 0
    numedges = np.count_nonzero(nf)
    numvertices = nf.shape[0] - 2
    numslacks = numedges
    slack_counter = 0
    edge_counter = 0

    primal_constraints = np.zeros(
        (numedges + numvertices + 2, numedges + numslacks + 1))

    obj = np.zeros(numedges + numslacks)

    for i in range(numvertices + 2):
        for j in range(numvertices + 2):
            if nf[i, j] != 0:
                if i == 0:
                    obj[edge_counter] = -1
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter,
                                       numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + j, edge_counter] = 1
                    edge_counter += 1
                    slack_counter += 1
                elif j == numvertices + 1:
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter,
                                       numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + i, edge_counter] = -1
                    edge_counter += 1
                    slack_counter += 1
                else:
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter,
                                       numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + i, edge_counter] = -1
                    primal_constraints[numedges + j, edge_counter] = 1
                    edge_counter += 1
                    slack_counter += 1
    
    primal_constraints = np.vstack((primal_constraints[:numedges], primal_constraints[numedges+1:numedges+numvertices+1]))
    
    sign_constraints = np.hstack(
        (np.eye(2 * numedges), np.zeros(2 * numedges).reshape(1, 2 * numedges).T))

    LPMatrix = np.vstack((primal_constraints, sign_constraints))

    return LPMatrix, obj


In [3]:
class LPSolution(object):

    def __init__(self, num_vars=0, var_vals=list(), obj=0):

        self.num_vars = num_vars
        self.obj = obj
        self.var_vals = var_vals

    def __str__(self):

        sol = ""
        sol += "\tSolution to the LP is as follows:\n\n"
        sol += "optim\t:=\t" + str(self.obj) + "\n\n"
        for i in range(self.num_vars):
            if i in self.var_vals:
                sol += "x_" + str(i + 1) + "*\t:=\t" + \
                    str(self.var_vals[i]) + "\n"
            else:
                sol += "x_" + str(i + 1) + "*\t:=\t" + str(0.0) + "\n"
        return sol

In [4]:
class PrimalDual(object):
    
    def __init__(self, num_eq_constraints, num_vars, c, constraints=None, tol=10**-20, A=None, b=None):
        self.num_eq_constraints = num_eq_constraints
        self.num_vars = num_vars
        self.A = A
        self.b = b
        self.tol = tol
        self.constraints = constraints
        if constraints is not None:
            self.A = constraints[:self.num_eq_constraints, :-1]
            self.b = constraints[:self.num_eq_constraints, -1]
        self.c = c
        if A is not None:
            self.num_vars = A.shape[1]
        self.x = None
        self.s = None
        self.lamda = None
        self.obj = 0
        
    def fetch_constraints(self, filename):
        constraints = np.loadtxt(filename)
        self.A = constraints[:self.num_eq_constraints, :-1]
        self.b = constraints[:self.num_eq_constraints, -1]
    
    def run_PDBarrier(self):
        
        if self.A is None or self.b is None:
            raise RuntimeError("Please fetch the constraints first!")
        
        A = self.A.copy()
        b = self.b.copy()
        c = self.c.copy()
        
        
        np.random.seed(2010)
        x = np.random.random(A.shape[1])
        lamda = np.random.random(A.shape[0])
        s = np.random.random(A.shape[1])
        
        n = A.shape[1]
        mu = np.dot(x, s) / n
        sigma = 1 - (1 / np.sqrt(n))
        e = np.ones(x.shape[0])
        eta = 0.9999
        
        while mu > self.tol:
            X = np.diag(x)
            S = np.diag(s)
            I = np.eye(x.shape[0])
            bmatrix = np.block([
                [np.zeros((A.shape[1], A.shape[1])), A.T, I],
                [A, np.zeros((A.shape[0], A.shape[0])), np.zeros((A.shape[0], x.shape[0]))],
                [S, np.zeros((S.shape[0], A.shape[0])), X]
            ])
            rhs = -1 * np.hstack((
                np.dot(A.T, lamda) + s - c,
                np.dot(A, x) - b,
                np.dot(np.dot(X, S), e) - (sigma * mu * e)
            ))
            delta = np.linalg.solve(bmatrix, rhs)
            delx = delta[:x.shape[0]]
            dell = delta[x.shape[0]:-s.shape[0]]
            dels = delta[-s.shape[0]:]
            alphax = 1.0

            if delx[delx < 0].shape[0] > 0:
                alphax = np.minimum(1.0, eta * np.min(-1 * x[delx < 0] / delx[delx < 0]))

            alphas = 1.0
            if dels[dels < 0].shape[0] > 0:
                alphas = np.minimum(1.0, eta * np.min(-1 * s[dels < 0] / dels[dels < 0]))
            x = x + (alphax * delx)
            s = s + (alphas * dels)
            lamda = lamda + (alphas * dell)
            mu = np.dot(s, x) / n
        
        self.x = x.copy()
        self.s = s.copy()
        self.lamda = lamda.copy()
        
#         self.x[self.x <= self.tol] = 0
#         self.s[self.s <= self.tol] = 0
#         self.lamda[self.lamda <= self.tol] = 0
        
        self.obj = np.dot(c, x)
        
        var_vals = {i: self.x[i] for i in range(self.num_vars)}
        return LPSolution(self.num_vars, var_vals, self.obj)
        

In [15]:
constraints, c = nf2PrimalLP("../assn3/nf1.dat")
pd = PrimalDual(14, 10, c, constraints, 10**-35)
sol = pd.run_PDBarrier()
print(sol)

	Solution to the LP is as follows:

optim	:=	-26.0

x_1*	:=	13.0
x_2*	:=	13.0
x_3*	:=	3.3004988936168926
x_4*	:=	12.0
x_5*	:=	2.3004988936168926
x_6*	:=	14.0
x_7*	:=	8.845268048775033e-36
x_8*	:=	19.0
x_9*	:=	7.0
x_10*	:=	7.0



In [18]:
constraints, c = nf2PrimalLP("../assn3/nf2.dat")
pd = PrimalDual(39, 58, c, constraints, 10**-30)

In [19]:
sol = pd.run_PDBarrier()

In [20]:
print(sol)

	Solution to the LP is as follows:

optim	:=	-29.0

x_1*	:=	11.0
x_2*	:=	8.0
x_3*	:=	10.0
x_4*	:=	11.09449633871313
x_5*	:=	3.482551770684198
x_6*	:=	3.0
x_7*	:=	5.0
x_8*	:=	5.166381308638481
x_9*	:=	2.265478443006163
x_10*	:=	8.328752432830271
x_11*	:=	0.7606121844749162
x_12*	:=	5.724299446081947
x_13*	:=	1.048547568055392
x_14*	:=	1.7599904027736808
x_15*	:=	9.334505935939449
x_16*	:=	0.5770481093973272
x_17*	:=	0.6070874872000932
x_18*	:=	2.2984161740867783
x_19*	:=	1.0008421263890366
x_20*	:=	4.6358775000183075
x_21*	:=	3.0249121697150576
x_22*	:=	2.3273039665766713
x_23*	:=	2.999157873610963
x_24*	:=	4.370863400577794
x_25*	:=	3.5020302440147955
x_26*	:=	5.827830216547376
x_27*	:=	8.14255321941038
x_28*	:=	0.4939093909288522
x_29*	:=	12.729696395911374
x_30*	:=	9.860761315262648e-31
x_31*	:=	6.999999999999999
x_32*	:=	9.675872040601473e-31
x_33*	:=	6.905503661286868
x_34*	:=	0.5174482293158019
x_35*	:=	7.888609052210118e-31
x_36*	:=	1.0908467205009304e-30
x_37*	:=	0.8336186913615

In [25]:
constraints, c = nf2DualLP("../assn3/nf1.dat")
pd = PrimalDual(10, 24, c, constraints, 10**-35)
sol = pd.run_PDBarrier()
print(sol)

	Solution to the LP is as follows:

optim	:=	26.0

x_1*	:=	2.8735742344691386e-36
x_2*	:=	5.989383377505233e-17
x_3*	:=	1.7241445406814834e-36
x_4*	:=	1.0
x_5*	:=	5.953510223772712e-18
x_6*	:=	6.437432320626496e-20
x_7*	:=	9.578580781563795e-37
x_8*	:=	1.7465248547516345e-35
x_9*	:=	1.0
x_10*	:=	1.0
x_11*	:=	1.0
x_12*	:=	1.0
x_13*	:=	1.7023305412637085e-35
x_14*	:=	1.0
x_15*	:=	6.6313251564672425e-37
x_16*	:=	6.6313251564672425e-37
x_17*	:=	1.7241445406814827e-36
x_18*	:=	7.183935586172846e-37
x_19*	:=	2.1551806758518538e-36
x_20*	:=	6.15765907386244e-37
x_21*	:=	1.0
x_22*	:=	4.4194313487925974e-37
x_23*	:=	1.231531814772488e-36
x_24*	:=	1.231531814772488e-36



In [26]:
constraints, c = nf2DualLP("../assn3/nf2.dat")
pd = PrimalDual(29, 68, c, constraints, 10**-30)
sol = pd.run_PDBarrier()
print(sol)

	Solution to the LP is as follows:

optim	:=	29.0

x_1*	:=	1.0
x_2*	:=	1.330185270178119e-31
x_3*	:=	1.0
x_4*	:=	1.34240331292824e-31
x_5*	:=	2.7447794762824267e-30
x_6*	:=	1.0
x_7*	:=	1.0
x_8*	:=	1.162847857919234e-30
x_9*	:=	1.318713032783521e-30
x_10*	:=	3.7621242634543086e-31
x_11*	:=	2.850840374296305e-31
x_12*	:=	8.455212988199388e-32
x_13*	:=	1.8734546745408097e-31
x_14*	:=	7.559511696031214e-31
x_15*	:=	2.7918840931687135e-31
x_16*	:=	8.00093975876861e-32
x_17*	:=	2.570251283211861e-31
x_18*	:=	1.441653550129601e-31
x_19*	:=	3.0794181731496263e-31
x_20*	:=	2.198529380824206e-31
x_21*	:=	1.3331671378051388e-30
x_22*	:=	2.7125430587870283e-30
x_23*	:=	9.537522504540348e-31
x_24*	:=	1.9033731500853237e-30
x_25*	:=	3.258565188304688e-30
x_26*	:=	1.1766797414474958e-30
x_27*	:=	2.445659566931115e-30
x_28*	:=	5.8490198893131126e-31
x_29*	:=	1.3759314233336792e-30
x_30*	:=	2.5705571366951317e-30
x_31*	:=	1.0
x_32*	:=	3.2748629938418195e-30
x_33*	:=	2.291103753713132e-30
x_34*	:=	2.520